In [63]:
import os
from PIL import Image
folder_path="/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/hazelnut/train/good/"
path_os=os.listdir(folder_path)
img_path=[folder_path+i for i in path_os]
PIL_image=[Image.open(i).convert("RGB") for i in img_path]

from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
from transformers import AutoImageProcessor, AutoModel
model_name = "facebook/dino-vitb8"
processor = AutoImageProcessor.from_pretrained(model_name)
encoder = AutoModel.from_pretrained(model_name)
encoder.eval()

import torch

normal_embeddings = []

for img in PIL_image:
    x = transform(img)

    with torch.no_grad():
        inputs = processor(images=x, return_tensors="pt", do_rescale=False)
        outputs = encoder(**inputs)

    embedding = outputs.last_hidden_state[:, 0, :]   # CLS token
    embedding = embedding.squeeze(0)                 # shape: (768,)
    normal_embeddings.append(embedding)

from sklearn.neighbors import NearestNeighbors
import numpy as np

# Convert list of tensors → 2D numpy array
normal_embeddings_np = torch.stack(normal_embeddings).numpy()

knn_hazelnut = NearestNeighbors(n_neighbors=5)
knn_hazelnut.fit(normal_embeddings_np)

import joblib

joblib.dump(knn, "knn_hazelnut_good_model.pkl")
print("Saved KNN model!")

# --- Load a Good-test image ---
test_path = "/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/hazelnut/test/good/000.png"
img = Image.open(test_path).convert("RGB")

# --- Preprocess ---
x = transform(img)

# --- Get test embedding ---
with torch.no_grad():
    inputs = processor(images=x, return_tensors="pt", do_rescale=False)
    outputs = encoder(**inputs)

test_embed = outputs.last_hidden_state[:, 0, :].squeeze(0)  # (768,)

# --- Convert to numpy ---
test_embed_np = test_embed.numpy().reshape(1, -1)

# --- Compute distance using k-NN ---
dist, _ = knn.kneighbors(test_embed_np)

anomaly_score = dist.mean()
print("Anomaly Score:", anomaly_score)
# --- Load a defective-test image ---
test_path = "/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/hazelnut/test/crack/000.png"
img = Image.open(test_path).convert("RGB")

# --- Preprocess ---
x = transform(img)

# --- Get test embedding ---
with torch.no_grad():
    inputs = processor(images=x, return_tensors="pt", do_rescale=False)
    outputs = encoder(**inputs)

test_embed = outputs.last_hidden_state[:, 0, :].squeeze(0)  # (768,)

# --- Convert to numpy ---
test_embed_np = test_embed.numpy().reshape(1, -1)

# --- Compute distance using k-NN ---
dist, _ = knn.kneighbors(test_embed_np)

anomaly_score = dist.mean()
print("Anomaly Score:", anomaly_score)
import os
import numpy as np
import torch
from PIL import Image
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_fscore_support
import pandas as pd

# allowed image extensions
valid_ext = [".png", ".jpg", ".jpeg", ".bmp"]

# paths
test_root = "/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/hazelnut/test/"
good_dir = os.path.join(test_root, "good")

# anomalies: any folder except 'good'
anomaly_dirs = [
    os.path.join(test_root, d)
    for d in os.listdir(test_root)
    if d != "good" and os.path.isdir(os.path.join(test_root, d))
]

# ---------- PRINT PATHS ----------
print("Good dir:", good_dir)
print("Anomaly dirs:", anomaly_dirs)

print("\nFiles in good:", os.listdir(good_dir))
for ad in anomaly_dirs:
    print("Files in", ad, ":", os.listdir(ad))


# helper to get embedding (returns numpy 1x768)
def get_embed_from_pil(img_pil):
    x = transform(img_pil)  # transform defined earlier
    with torch.no_grad():
        inputs = processor(images=x, return_tensors="pt", do_rescale=False)
        outputs = encoder(**inputs)
    emb = outputs.last_hidden_state[:, 0, :].squeeze(0)  # (768,)
    return emb.cpu().numpy().reshape(1, -1)


# evaluate images
scores = []
labels = []
paths = []

count_good = 0
count_anomaly = 0


# -------------------------
# GOOD images (label = 0)
# -------------------------
for fn in sorted(os.listdir(good_dir)):
    if not any(fn.lower().endswith(ext) for ext in valid_ext):
        continue

    p = os.path.join(good_dir, fn)

    try:
        img = Image.open(p).convert("RGB")
    except:
        continue

    emb_np = get_embed_from_pil(img)
    dist, _ = knn.kneighbors(emb_np)
    score = dist.mean()

    scores.append(float(score))
    labels.append(0)
    paths.append(p)

    count_good += 1


# -------------------------
# ANOMALY images (label = 1)
# -------------------------
for ad in anomaly_dirs:
    for fn in sorted(os.listdir(ad)):
        if not any(fn.lower().endswith(ext) for ext in valid_ext):
            continue

        p = os.path.join(ad, fn)

        try:
            img = Image.open(p).convert("RGB")
        except:
            continue

        emb_np = get_embed_from_pil(img)
        dist, _ = knn.kneighbors(emb_np)
        score = dist.mean()

        scores.append(float(score))
        labels.append(1)
        paths.append(p)

        count_anomaly += 1


# ---------- PRINT COUNTS ----------
print("\nGood images processed:", count_good)
print("Anomaly images processed:", count_anomaly)
print("Total processed:", count_good + count_anomaly)


# convert to numpy
scores = np.array(scores)
labels = np.array(labels)

# -------------------------
# Compute performance metrics
# -------------------------
roc = roc_auc_score(labels, scores)
pr = average_precision_score(labels, scores)

# find best threshold by maximizing F1
thresholds = np.linspace(scores.min(), scores.max(), 200)
best_f1 = 0.0
best_thresh = thresholds[0]
best_prec = best_rec = 0.0

for t in thresholds:
    preds = (scores >= t).astype(int)
    prec, rec, f1, _ = precision_recall_fscore_support(
        labels, preds, average='binary', zero_division=0
    )
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t
        best_prec = prec
        best_rec = rec

# ---------- PRINT METRICS ----------
print("\nAUC-ROC:", round(roc, 4))
print("AUC-PR :", round(pr, 4))
print(f"Best F1: {best_f1:.4f} at threshold {best_thresh:.4f}")
print(f"Precision: {best_prec:.4f}, Recall: {best_rec:.4f}")


# save results
df = pd.DataFrame({"path": paths, "label": labels, "score": scores})
df.to_csv("hazelnut_test_scores_knn.csv", index=False)
print("\nSaved hazelnut_test_scores_knn.csv")

Some weights of ViTModel were not initialized from the model checkpoint at facebook/dino-vitb8 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Saved KNN model!
Anomaly Score: 23.142659759521486
Anomaly Score: 52.44691390991211
Good dir: /Users/ggharish13/Data Science/Capstone Project/Final Project/Images/hazelnut/test/good
Anomaly dirs: ['/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/hazelnut/test/crack', '/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/hazelnut/test/print', '/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/hazelnut/test/cut', '/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/hazelnut/test/hole']

Files in good: ['001.png', '000.png']
Files in /Users/ggharish13/Data Science/Capstone Project/Final Project/Images/hazelnut/test/crack : ['002.png', '016.png', '017.png', '003.png', '015.png', '001.png', '000.png', '014.png', '010.png', '004.png', '005.png', '011.png', '007.png', '013.png', '012.png', '006.png', '008.png', '009.png']
Files in /Users/ggharish13/Data Science/Capstone Project/Final Project/Images/hazelnut/test/print : 

In [64]:
import torch
from PIL import Image
import numpy as np


def predict_image(path):
    # load image
    img = Image.open(path).convert("RGB")
    
    # get embedding
    emb_np = get_embed_from_pil(img)   # function defined earlier
    
    # kNN distance
    dist, _ = knn.kneighbors(emb_np)
    score = dist.mean()

    # classification
    if score < 25.1:
        label = "GOOD"

    elif 25.1 <= score < 27.5:
        label = "CUT"

    elif 27.5 <= score <= 37.5:
        label = "CUT or HOLE"    # overlap area

    elif 37.5 <= score <= 47.1:
        label = "CRACK or HOLE"    # overlap area

    elif score > 47.1:
        label = "CRACK"


    return score, label

img_path = "/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/hazelnut/test/good/000.png"

score, label = predict_image(img_path)
print("Score:", score)
print("Prediction:", label)


Score: 23.142659759521486
Prediction: GOOD


In [65]:
img_path = "/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/hazelnut/test/crack/001.png"

score, label = predict_image(img_path)
print("Score:", score)
print("Prediction:", label)


Score: 42.9609489440918
Prediction: CRACK or HOLE


In [66]:
import os
from PIL import Image

cut_score_list=[]
cut_label_list=[]
folder_path = "/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/hazelnut/test/cut/"
for filename in os.listdir(folder_path):
    if filename.lower().endswith((".png", ".jpg", ".jpeg")):
        img_path = os.path.join(folder_path, filename)
        score, label = predict_image(img_path)
        cut_score_list.append(score)
        label= 1 if label in ["CUT", "CUT or HOLE"] else 0
        cut_label_list.append(label)

crack_score_list=[]
crack_label_list=[]
folder_path = "/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/hazelnut/test/crack/"
for filename in os.listdir(folder_path):
    if filename.lower().endswith((".png", ".jpg", ".jpeg")):
        img_path = os.path.join(folder_path, filename)
        score, label = predict_image(img_path)
        crack_score_list.append(score)
        label= 1 if label in ["CUT", "CRACK or HOLE"] else 0
        crack_label_list.append(label)

good_score_list=[]
good_label_list=[]
folder_path = "/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/hazelnut/test/good/"
for filename in os.listdir(folder_path):
    if filename.lower().endswith((".png", ".jpg", ".jpeg")):
        img_path = os.path.join(folder_path, filename)
        score, label = predict_image(img_path)
        good_score_list.append(score)
        label=1 if label=="GOOD" else 0
        good_label_list.append(label)

hole_score_list=[]
hole_label_list=[]
folder_path = "/Users/ggharish13/Data Science/Capstone Project/Final Project/Images/hazelnut/test/hole/"
for filename in os.listdir(folder_path):
    if filename.lower().endswith((".png", ".jpg", ".jpeg")):
        img_path = os.path.join(folder_path, filename)
        score, label = predict_image(img_path)
        hole_score_list.append(score)
        label=1 if label=="CUT or HOLE" else 0
        hole_label_list.append(label)


In [67]:
print("cut_score_list min=",min(cut_score_list),"cut_score_list max=",max(cut_score_list))
print("crack_score_list min=",min(crack_score_list),"crack_score_list max=",max(crack_score_list))
print("good_score_list min=",min(good_score_list),"good_score_list max=",max(good_score_list))
print("hole_score_list min=",min(hole_score_list),"hole_score_list max=",max(hole_score_list))

cut_score_list min= 26.14229965209961 cut_score_list max= 38.96882781982422
crack_score_list min= 31.095760345458984 crack_score_list max= 61.11775894165039
good_score_list min= 23.142659759521486 good_score_list max= 25.0428165435791
hole_score_list min= 27.51972312927246 hole_score_list max= 47.1120246887207
